In [ ]:
%matplotlib inline
from __future__ import print_function
import datetime
import os
import sys
import copy
import random
import traceback
import ipywidgets
import numpy as np
import tensorflow as tf
from IPython.display import display
from six.moves import cPickle as pickle

import outputer
import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

In [ ]:
pickle_file = '../ud730/notMNIST_full.pickle'

train_dataset = []
train_labels = []
test_dataset = []
test_labels = []

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def setup_data(inputs_train, labels_train, inputs_test, labels_test):
    data = {
        "image_size": 28,
        "label_count": 10,
        "channel_count": 1
    }
    data["total_image_size"] = data["image_size"] * data["image_size"]

    def setup_data(inputs, labels, name):
        inputs = inputs.reshape((-1, data["image_size"], data["image_size"], data["channel_count"])).astype(np.float32)
        # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
        labels = (np.arange(data["label_count"]) == labels[:,None]).astype(np.float32)
        print(name + " set", inputs.shape, labels.shape)
        return inputs, labels
    data["train"], data["train_labels"] = setup_data(inputs_train, train_labels, "Training")
    data["test"], data["test_labels"] = setup_data(inputs_test, labels_test, "Test")
    return data

full_data = setup_data(train_dataset, train_labels, test_dataset, test_labels)
print(full_data.keys())

In [ ]:
del train_dataset
del train_labels
del test_dataset
del test_labels

In [ ]:
def setup_validate(data, train_count, validate_count, seed=None):
    if seed:
        np.random.seed(seed)

    def randomize(inputs, labels):
        permutation = np.random.permutation(labels.shape[0])
        shuffled_inputs = inputs[permutation,:,:,:]
        shuffled_labels = labels[permutation,:]
        return shuffled_inputs, shuffled_labels

    train_inputs = data["train"][:]
    train_labels = data["train_labels"][:]
    cross_data = copy.copy(data)

    train_inputs, train_labels = randomize(train_inputs, train_labels)
    cross_data["train"] = train_inputs[:train_count]
    cross_data["train_labels"] = train_labels[:train_count]

    cross_data["valid"] = train_inputs[train_count:train_count + validate_count]
    cross_data["valid_labels"] = train_labels[train_count:train_count + validate_count]
    return cross_data
cross_data = setup_validate(full_data, 1000, 100)
print(cross_data["train_labels"].shape)
print(cross_data["train_labels"][0])
print(full_data["train_labels"][0])
print(cross_data["valid"].shape)
del cross_data

In [ ]:
def graph_input_shape(batch_size, data):
    image_size = data["image_size"]
    channel_count = data["channel_count"]
    return (batch_size, image_size, image_size, channel_count)

def graph_output_shape(batch_size, data):
    return (batch_size, data["label_count"])

def setup_graph(batch_size, data, stack):
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        input_shape = graph_input_shape(batch_size, data)
        output_shape = graph_output_shape(batch_size, data)
        train = tf.placeholder(tf.float32, shape=input_shape)
        labels= tf.placeholder(tf.float32, shape=output_shape)
        verify= tf.placeholder(tf.float32, shape=input_shape)

        layers = stack.construct(input_shape, output_shape)
        l2_loss = convnet.setup_layers(layers)
        
        logits        = convnet.connect_model(train,  layers, True)[-1]
        verify_logits = convnet.connect_model(verify, layers, False)[-1]
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_loss
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": stack.construct_optimizer(loss),

            # Predictions for the training, validation, and test data.
            "predictions": tf.nn.softmax(logits),
            "verify": verify,
            "verify_predictions": tf.nn.softmax(verify_logits),
            "saver": tf.train.Saver()
        }
    return info

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
def batch_accuracy(session, graph_info, inputs, labels, batch_size):
    total_accuracy = 0
    batch_count = len(inputs) / batch_size
    for b in xrange(batch_count):
        batch_data = inputs[b * batch_size: (b + 1) * batch_size]
        feed_dict = { graph_info["verify"] : batch_data }
        predictions = session.run([graph_info["verify_predictions"]], feed_dict=feed_dict)[0]
        total_accuracy += accuracy(predictions, labels[b * batch_size: (b + 1) * batch_size]) / float(batch_count)
    return total_accuracy

In [ ]:
def run_graph(
    graph_info,
    data,
    step_count,
    report_every=50,
    verbose=True,
    accuracy_minimum=None,
    progress=None,
    tracker=None
):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        
        # Optionally restore graph parameters from disk.
        convnet.restore_model(graph_info, session)
        
        batch_size = graph_info["batch_size"]
        valid_accuracy = 0
        try:
            for step in xrange(step_count + 1):
                # Update progress bar, if present
                if progress:
                    progress.value = step

                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (data["train_labels"].shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = data["train"][offset:(offset + batch_size), :, :, :]
                batch_labels = data["train_labels"][offset:(offset + batch_size), :]
                
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                targets = [graph_info["optimizer"], graph_info["loss"], graph_info["predictions"]]
                feed_dict = {graph_info["train"] : batch_data, graph_info["labels"] : batch_labels}
                
                _, loss, predictions = session.run(targets, feed_dict=feed_dict)
                
                reporting = step % report_every == 0
                if reporting or tracker:
                    batch_score = (loss, accuracy(predictions, batch_labels))
                    if tracker:
                        tracker(batch_score)
                    
                if np.isnan(loss):
                    print("Error computing loss")
                    return valid_accuracy
                
                if reporting:
                    if verbose:
                        print("Minibatch loss at step", step, ":", loss)
                        print("Minibatch accuracy: %.1f%%" % batch_score[1])
                    valid_accuracy = batch_accuracy(
                        session, graph_info, data["valid"], data["valid_labels"], batch_size
                    )
                    print("Validation accuracy: %.1f%%" % valid_accuracy)
                    if accuracy_minimum and step > 0 and valid_accuracy < accuracy_minimum:
                        print("Early out.")
                        break
            if verbose:
                test_accuracy = batch_accuracy(
                    session, graph_info, data["test"], data["test_labels"], batch_size
                )
                print("Test accuracy: %.1f%%" % test_accuracy)
            return valid_accuracy
        finally:
            # Optionally save out graph parameters to disk.
            convnet.save_model(graph_info, session)

In [ ]:
results_path = outputer.setup_directory("temp", "notMNIST_results")

def save_results(timestamp, results):
    with open(os.path.join(results_path, timestamp + ".csv"), "w") as text_file:
        text_file.write("Loss,Accuracy\n")
        for score in results:
            text_file.write((",".join(str(v) for v in score)) + "\n")
    print("Saved results:", timestamp)

In [ ]:
def make_eval(
    batch_size=16,
    eval_steps=100000,
    valid_steps=5000
):
    # Set up to show a progress bar so you some mesure of time required. Updated in run_graph above.
    progress_bar = ipywidgets.FloatProgress(min=0, max=eval_steps, description="Graph Steps:")
    display(progress_bar)
    
    # Set up to show current training results as well as a running average. updated in record_score below. 
    def setup_label(title):
        return ipywidgets.FloatText(value=0, description=title, disabled=True)
    current_display = [setup_label(title) for title in ["Loss", "Accuracy"]]
    average_display = [setup_label(" ") for _ in current_display]
    display(ipywidgets.HBox([
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Current</div>")] + current_display),
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Running Average</div>")] + average_display)
    ]))
    
    def evaluate(stack, entropy):   
        data = setup_validate(full_data, eval_steps, valid_steps)

        try:
            evo_graph = setup_graph(batch_size, data, stack)
        except KeyboardInterrupt:
            raise
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, "temp", outputer.timestamp("ERR~", "txt"))
            return -10

        timestamp = outputer.timestamp()
        with open(os.path.join(results_path, timestamp + ".xml"), "w") as text_file:
            text_file.write(convevo.serialize(stack))

        # Record and display the results
        results = []
        def record_score(score):
            results.append(score)
            for display, value in zip(current_display, score):
                display.value = value
            
            resultCount = min(len(results), 100)
            averages = [sum(x)/resultCount for x in zip(*results[-resultCount:])]
            for display, value in zip(average_display, averages):
                display.value = value
                
        convnet.setup_save_model(evo_graph, os.path.join(results_path, timestamp + ".ckpt"))
        
        try:
            return run_graph(
                evo_graph,
                data,
                eval_steps,
                report_every=eval_steps/4,
                verbose=False,
                accuracy_minimum=50,
                progress=progress_bar,
                tracker=record_score
            )
        except KeyboardInterrupt:
            raise
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, results_path, timestamp + "~ERR.txt")
            return -1
        finally:
            save_results(timestamp, results)
    return evaluate

In [ ]:
def create_stack(patch_size, stride, depth, hidden_size, label_count, init_scale, optimizer_name=None):
    if optimizer_name:
        optimizer = convevo.Optimizer(optimizer_name, 0.05)
        optimizer.default_parameters()
    else:
        optimizer = None
        
    conv_layers = [
        ("conv_bias", patch_size, stride, depth, "SAME", True),
        ("conv_bias", patch_size, stride, depth, "SAME", True)
    ]

    return convevo.create_stack(conv_layers, [], True, [hidden_size, label_count], 0.0, init_scale, 0.0, optimizer)

In [ ]:
def test_optimizers():
    for optimizer_name in ["GradientDescent", "Adadelta", "Adagrad", "Momentum", "Adam", "RMSProp"]:
        # As of this writing "Ftrl" is not supported on the GPU
        batch_size = 16
        eval_steps = 10000
        test_stack = create_stack(5, 2, 64, 128, 10, 0.1, optimizer_name)
        test_data = setup_validate(full_data, eval_steps * batch_size, 500)
        test_graph = setup_graph(batch_size, test_data, test_stack)
        print (run_graph(test_graph, test_data, eval_steps, report_every=eval_steps/4, verbose=False))
test_optimizers()

In [ ]:
with outputer.TeeOutput(os.path.join(results_path, outputer.timestamp("Eval", "txt"))):
    test_stack = create_stack(5, 2, 64, 128, 10, 0.1)
    print(convevo.serialize(test_stack))
    make_eval()(test_stack, random.Random(42))

In [ ]:
mutate_entropy = random.Random(42)
scored_mutants = []
breed_options = {
    "input_shape": graph_input_shape(16, full_data),
    "output_shape": graph_output_shape(16, full_data)
}
mutator = darwin.Darwin(convevo.serialize, lambda s,e: 0, convevo.breed)
prototypes = [create_stack(5, 2, 64, 64, 10, 0.1)]
for mutant in mutator.init_population(prototypes, 20, False, breed_options, mutate_entropy):
    scored_mutants.append((mutant, 0.0))
    
convevo.output_results(scored_mutants, "temp", "mutants.xml", 0, 0)

In [ ]:
mutate_entropy = random.Random(42)
mutant_children = []
for _ in range(20):
    mutant_a = mutate_entropy.choice(scored_mutants)[0]
    mutant_b = mutate_entropy.choice(scored_mutants)[0]
    mutant_children.append((convevo.breed([mutant_a, mutant_b], breed_options, mutate_entropy), 0.0))
    
convevo.output_results(mutant_children, "temp", "mutant_offspring.xml", 42, 0)

In [ ]:
prototypes = [
    create_stack(5, 2, 64, 128, 10, 0.10, "GradientDescent"),
    create_stack(6, 2, 128, 64, 10, 0.05, "Adadelta"),
    create_stack(4, 2, 64, 128, 10, 0.10, "Adagrad"),
    create_stack(5, 1, 256,128, 10, 0.02, "Adam"),
    create_stack(2, 2, 64, 128, 10, 0.20, "RMSProp")
]

In [ ]:
with outputer.TeeOutput(os.path.join("temp", outputer.timestamp("notMNIST_Evolve_", "txt"))):
    mutate_seed = random.randint(1, 100000)
    print("Mutate Seed:", mutate_seed)
    mutate_entropy = random.Random(mutate_seed)
    eval_seed = random.randint(1, 100000)
    print("Eval Seed:", eval_seed)
    eval_entropy = random.Random(eval_seed)

    population_size = 20
    generations = 10
    
    breed_options = {
        "input_shape": graph_input_shape(16, full_data),
        "output_shape": graph_output_shape(16, full_data)
    }

    for stack in prototypes:
        stack.make_safe(breed_options["input_shape"], breed_options["output_shape"])

    charles = darwin.Darwin(convevo.serialize, make_eval(), convevo.breed)
    charles.init_population(prototypes, population_size, True, breed_options, mutate_entropy)

    for g in range(generations):
        print("Generation", g)
        results = charles.evaluate(eval_entropy)
        convevo.output_results(results, "temp", outputer.timestamp() + ".xml")
        charles.repopulate(population_size, 0.25, 4, results, breed_options, mutate_entropy)

In [ ]:
best = charles.best()
print("Best score:", best[1])
print(convevo.serialize(best[0]))

In [ ]:
results = darwin.descending_score(charles.history.values())
convevo.output_results(results, "testing", "notminist_full_evolve_run.xml", mutate_seed, eval_seed)
len(results)

In [ ]:
errors = []

for root, dirs, files in os.walk('temp'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        if low_name.startswith("err"):
            with open (path, "r") as error_file:
                lines=error_file.readlines()
                errors.append((path, lines[-1]))

for path, error in sorted(errors, key=lambda e: e[0]):
    print(path)
    print(error)

In [ ]:
failed = 0
print (convevo.serialize(charles.population[failed]))
make_eval()(charles.population[failed], eval_entropy)

In [ ]:
error_population,_,eval_seed = convevo.load_population("testing/error.xml")
print(len(error_population))

error_charles = darwin.Darwin(error_population, convevo.serialize, eval_stack, convevo.breed)
error_results = error_charles.evaluate(random.Random(eval_seed))